Python wrapper for the JTAG Atlantic communiation library

Adapted from http://alterawiki.com/wiki/High_Speed_Image_Download_Demo

In [ ]:
import ctypes

In [ ]:
# requiring jtag_atlantic.dll and jtag_client.dll put in the same folder as this notebook
dll_path = 'jtag_atlantic.dll'

In [ ]:
atlantic_dll = ctypes.cdll.LoadLibrary(dll_path)

# Function definitions

In [ ]:
"""
JTAGATLANTIC * jtagatlantic_open (
    const char * chain,
    int device_index,
    int link_instance,
    const char * app_name);
"""
JAOpen = getattr(atlantic_dll,'?jtagatlantic_open@@YAPEAUJTAGATLANTIC@@PEBDHH0@Z')  # mangled name obtained from dllwalker
JAOpen.argtypes = [ctypes.c_char_p, ctypes.c_int64, ctypes.c_int64, ctypes.c_char_p]
JAOpen.restype = ctypes.c_void_p

"""
int  jtagatlantic_get_error (const char * * other_info);
"""
JAGetError = getattr(atlantic_dll,'?jtagatlantic_get_error@@YA?AW4JATL_ERROR@@PEAPEBD@Z')
JAGetError.argtypes = [ctypes.c_void_p]
JAGetError.restype = ctypes.c_int64

"""
void jtagatlantic_close (JTAGATLANTIC * link);
"""
JAClose = getattr(atlantic_dll,'?jtagatlantic_close@@YAXPEAUJTAGATLANTIC@@@Z')
JAClose.argtypes = [ctypes.c_void_p]
JAClose.restype = None

"""
int  jtagatlantic_write (JTAGATLANTIC * link, const char * data, unsigned int count);
"""
JAWrite = getattr(atlantic_dll,'?jtagatlantic_write@@YAHPEAUJTAGATLANTIC@@PEBDI@Z')
JAWrite.argtypes = [ctypes.c_void_p, ctypes.c_char_p, ctypes.c_uint64]
JAWrite.restype = ctypes.c_int64

"""
int  jtagatlantic_flush (JTAGATLANTIC * link);
"""
JAFlush = getattr(atlantic_dll,'?jtagatlantic_flush@@YAHPEAUJTAGATLANTIC@@@Z')
JAFlush.argtypes = [ctypes.c_void_p]
JAFlush.restype = ctypes.c_int64

"""
int  jtagatlantic_read (JTAGATLANTIC * link, char * buffer, unsigned int buffsize);
"""
JARead = getattr(atlantic_dll,'?jtagatlantic_read@@YAHPEAUJTAGATLANTIC@@PEADI@Z')
JARead.argtypes = [ctypes.c_void_p, ctypes.c_char_p, ctypes.c_uint64]
JARead.restype = ctypes.c_int64

In [ ]:
# information obtained from running nios2-terminal.exe
cable_name = b"DE-SoC [USB-1]"  # related to the board you are using
device_index = 2  # related to the JTAG device on the board. 0: to let the DLL determine itself. 
                  #      1: usually the on-board FPGA "burner" chip, 2: the FPGA chip
link_instance = 0 # -1: to let the DLL determine the JTAG instance in the chip. 0: in my case I have only 1 instance
                  #      of JTAG-UART so it is indexed by 0.

# establish the link
link = JAOpen(
    ctypes.c_char_p(cable_name),
    ctypes.c_int64(device_index),
    ctypes.c_int64(link_instance),
    ctypes.c_char_p(b'')
)

# Load test image

In [ ]:
%pylab inline
import numpy as np
import skimage.io as skio
import skimage.transform as sktr

In [ ]:
img = skio.imread('./CAL_PRI_LOGO_ON_NAV_launch.png')
img = img[:,:,:3].sum(axis=2)
img = sktr.resize(img, (1024,1024))
img = (img != 0).astype(np.uint8)*255
img[-1,-1] = 255
img.dtype

In [ ]:
imshow(img.astype(np.float64))
colorbar()

In [ ]:
img2 = skio.imread('image347.jpg')
img2.shape, img2.dtype

In [ ]:
177-134

In [ ]:
img2 = img2[:-43,:,:].astype(float64).sum(axis=2)
img2 = sktr.resize(img2,(1024,1024), mode='constant')
img2 = (img2/img2.max()*255).astype(np.uint8)
img2[img2==254] = 255

In [ ]:
img_bytes = img.tobytes() + img2.tobytes()

In [ ]:
len(img_bytes)

In [ ]:
to_send = b'\x81' + img_bytes + b'\xFE\x01'
len(to_send)

# Send

In [ ]:
print('to send',len(to_send),'bytes')
JAFlush(link)
for i in range(0, len(to_send), 10000):
    to_send_chunk = to_send[i:i+10000]
    JAWrite(link, ctypes.c_char_p(to_send_chunk), len(to_send_chunk))
    JAFlush(link)

In [ ]:
JAClose(link)